Importing

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import kagglehub

path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sentiment140


In [ ]:
col = ['Target','Id','Time','Query','Name','Text']
data = pd.read_csv("/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv",names=col,encoding='ISO-8859-1')

In [ ]:
data['Target'].value_counts()

,count
Target,
0,800000
4,800000


In [ ]:
data.replace({'Target':{4:1}},inplace=True)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Stemming

In [ ]:
port = PorterStemmer()

In [ ]:
def stemming(data):
  content = re.sub('[^a-zA-Z]',' ',data)
  content = content.lower()
  content = content.split()
  content = [port.stem(i) for i in content if not i in stopwords.words('english')]
  content = " ".join(content)
  return content

In [ ]:
data['Keyword'] = data['Text'].apply(stemming)

In [ ]:
y = data['Target']
x = data['Keyword']

Splitting the data

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=4);

In [ ]:
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

Training

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

Prediction

In [ ]:
train_predict = model.predict(x_train)
train_acc = accuracy_score(y_train,train_predict)
print(train_acc)

0.80147578125


In [ ]:
test_predict = model.predict(x_test)
test_acc = accuracy_score(y_test,test_predict)
print(test_acc)

0.77811875


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.0 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import numpy as np

# Assuming 'model' is your trained Logistic Regression model
# Assuming 'vectorizer' is your fitted TfidfVectorizer
# Ensure 'stemming' function is defined as in your notebook

def predict_sentiment(text):
    # Apply the same preprocessing as training data
    processed_text = stemming(text)
    # Transform the text using the fitted vectorizer
    vectorized_text = vectorizer.transform([processed_text])
    # Predict the sentiment
    prediction = model.predict(vectorized_text)

    if prediction[0] == 1:
        return "<div style='background-color: lightgreen; padding: 10px;'>Positive</div>"
    else:
        return "<div style='background-color: salmon; padding: 10px;'>Negative</div>"

# Create the Gradio interface
interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=2, placeholder="Enter text here..."),
    outputs=gr.HTML(),  # Use HTML to display colored text
    title="Sentiment Analysis"
)

# Launch the interface
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f4b586b2752278335b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
